In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive

In [0]:
import numpy as np

In [0]:
# load data
# id=1QMue40RzXn8q87JM7M9KT40Z9cD1dqyU
X_TRAIN = drive.CreateFile({'id':'1QMue40RzXn8q87JM7M9KT40Z9cD1dqyU'}) # full arr
# id=1rYOqTxIYw0QUxh-w5OQYRHNsX-IeDmC9
Y_TRAIN = drive.CreateFile({'id':'1rYOqTxIYw0QUxh-w5OQYRHNsX-IeDmC9'})


X_TRAIN.GetContentFile('input_data.npy')
Y_TRAIN.GetContentFile('labels.npy')


In [0]:
INPUT_DATA = np.load('input_data.npy')
OUTPUT_DATA = np.load('labels.npy')

In [0]:

print(len(INPUT_DATA))
print(len(OUTPUT_DATA[123]))
OUTPUT_DATA.shape


In [0]:
####################################################
############# DATA LOADED SUCCESSFULLY##############
####################################################

In [0]:
from __future__ import print_function
import tensorflow as tf


In [0]:
# LOAD DATA

In [0]:
# Parameters
learning_rate = 0.000000000000000000000000000000001
num_steps = 500                       ###########################
batch_size = 128
display_step = 100

In [0]:
# Network Parameters
num_input = 16000 # 20K border pixels
n_hidden_1 = 16000 # 1st layer number of neurons
n_hidden_2 = 8000 # 2nd layer number of neurons
n_hidden_3 = 800
n_hidden_4 = 20
num_classes = 4 # 4 points are required to build a frame (total 5 frames) #########################


In [0]:
# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


In [0]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_hidden_4, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 2000 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    
    # Hidden fully connected layer with 200 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    
    
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    
    
    
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

In [0]:
# Construct model
logits = neural_net(X)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [0]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    step = 0
    for i in range(3000):
        
        
      
        a = INPUT_DATA[i].shape
        if(a[0]!=16000):
            continue
        
        batch_x = np.reshape(INPUT_DATA[i], (1,16000))
        batch_y = np.reshape(OUTPUT_DATA[i], (1,4))
        step = step + 1
        
        
        
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y}) ########################
        if i % 10 == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("--------------------------------------------------------")
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))


    print("Optimization Finished!")
    save_path = saver.save(sess, "/tmp/model.ckpt")

In [0]:
len(INPUT_DATA)

In [0]:
d = np.reshape(INPUT_DATA[0], (1,16000))
a, b = (d.shape)
print(a,b)

In [0]:
a = INPUT_DATA[0].shape
print(a[0])